# 从Titanic生存率预测问题入门Kaggle [u](https://zhuanlan.zhihu.com/p/53176091)


## 数据

下载数据。https://www.kaggle.com/c/titanic

其中

gender_submission.csv 文件记录真实的存活情况，1表示存活，0表示不幸遇难；  
test.csv 为测试数据集；  
train.csv 为训练数据集。

```
Age 年龄
Cabin 船舱号
Embarked 登船港口
Fare 票价
Name 乘客姓名
Parch 不同代直系亲属人数
SibSp 同代直系亲属人数
PassengerId 乘客ID
Pclass 客舱等级
Sex 性别
Ticket 船票编号
Survived 存活情况
```



In [107]:
#导入包
import numpy as np
import pandas as pd
import os

#导入数据
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

# 训练数据集大小： (891, 12)
# 测试数据集大小： (418, 11)
print('训练数据大小:',train.shape)
print('测试数据大小:',test.shape)

# 训练数据比预测数据多了一列：即标签"result"
print( train.columns )
print( test.columns )


#训练数据集比测试数据集多一列结果列
#现在将数据集进行合并，方便进行数据清洗和特征提取
full = pd.DataFrame()
full = pd.concat([train,test],ignore_index=True)  # 列数不一至的数据帧合并，缺失列的一方会添加空列(所有数据为空白)
print(full.columns)

full.head()
#查看描述统计和信息
full.describe()
#根据描述统计没有观察到异常值
full.info()
'''
通过info可以看出总共有1309条数据，其中Age，Cabin，Embarked，Fare，Survived均有不同程度的数据缺失，包含数值型数据（Age，Fare和Survived）
和字符型数值（Cabin和Embarked），这就需要我们根据不同的数据类型进行相应的数据预处理。
'''


currDir = os.getcwd() # os.path.dirname(os.path.abspath(__file__))
fname_full = os.path.join(currDir, 'full.csv')
full.to_csv(fname_full, index=False ,encoding="utf-8")

训练数据大小: (891, 12)
测试数据大小: (418, 11)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null 

## 数据清洗（Data Preparaion）

### 补全缺失值

```
对于数值型数据(Age,Fare)缺失值，我们采用平均值来填充；
对于分类数据(Cabin,Embarked)，我们采用众数填充；
```



In [108]:
from pandas import DataFrame
full['Age']  = full['Age'].fillna(full['Age'].mean())
#年龄因为不可从其他途径得知，采用平均年龄填充，此步骤对最终预测结果有影响。
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [109]:
"""
船票价格
"""
full['Fare'] = full['Fare'].fillna(full['Fare'].mean()) #船票价格用平均值填充
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [110]:
"""
登船港口
"""
full['Embarked'].value_counts() #查看登船港口的分布

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [111]:
full['Embarked'] = full['Embarked'].fillna('S') #S港登船的人最多，对空值使用S港填充

In [112]:
full['Cabin'].head()

0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object

In [113]:
full['Cabin']=full['Cabin'].fillna('U')#船舱号Cabin的缺失值比较多，我们对于空值填充U（Unknow）
full.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,U,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,U,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,U,S
5,6,0.0,3,"Moran, Mr. James",male,29.881138,0,0,330877,8.4583,U,Q
6,7,0.0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S
7,8,0.0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,U,S
8,9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,U,S
9,10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,U,C


## 特征提取

In [114]:
#将男女这种简单的类比区分直接用数值代替，‘male’为1，‘female’为0
full['Sex']=full['Sex'].map(lambda x: 1 if x=="male" else 0)#Series的map方法可以接受一个函数或含有映射关系的字典型对象
full['Sex'].head()

0    1
1    0
2    0
3    0
4    1
Name: Sex, dtype: int64

### One hot encode

In [115]:
embarkedDf = pd.get_dummies(full['Embarked'],prefix='Embarked')#predix参数用于指定类别标签
embarkedDf.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [116]:
full = pd.concat([full,embarkedDf],axis=1)
full.drop('Embarked',axis=1,inplace=True)#删除‘Embarked’列
full.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,U,0,0,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,U,0,0,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0,0,1
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,U,0,0,1


In [117]:
# 客舱等级（Pcalss）
pclassDf = pd.get_dummies(full['Pclass'],prefix='Pclass')
full = pd.concat([full,pclassDf],axis=1)
full.drop('Pclass',axis=1,inplace=True)
full.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,1,0.0,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,U,0,0,1,0,0,1
1,2,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1,0,0,1,0,0
2,3,1.0,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,U,0,0,1,0,0,1
3,4,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0,0,1,1,0,0
4,5,0.0,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,U,0,0,1,0,0,1


In [118]:
def get_title(name):#定义称谓提取函数
    str1=name.split(',')[1]#提取，后边的字符串
    str2=str1.split('.')[0]#提取.前面的字符串
    str3=str2.strip()#用于去除空格
    return str3
titleDf = pd.DataFrame()
titleDf['Title'] = full['Name'].map(get_title)
titleDf['Title'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Major             2
Ms                2
Mlle              2
Dona              1
Capt              1
Lady              1
Mme               1
Sir               1
Jonkheer          1
the Countess      1
Don               1
Name: Title, dtype: int64

In [119]:
#头衔类别有很多，我们需要对其进行进一步的分类映射
title_mapDict = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"
                    }
titleDf['Title'] = titleDf['Title'].map(title_mapDict)
titleDf['Title'].value_counts()

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64

In [120]:
titleDf = pd.get_dummies(titleDf['Title'])
titleDf.head()

,Master,Miss,Mr,Mrs,Officer,Royalty
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,1,0,0
4,0,0,1,0,0,0


In [121]:
full = pd.concat([full,titleDf],axis=1)
full.drop('Name',axis=1,inplace=True)
full.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,...,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Master,Miss,Mr,Mrs,Officer,Royalty
0,1,0.0,1,22.0,1,0,A/5 21171,7.2500,U,0,...,1,0,0,1,0,0,1,0,0,0
1,2,1.0,0,38.0,1,0,PC 17599,71.2833,C85,1,...,0,1,0,0,0,0,0,1,0,0
2,3,1.0,0,26.0,0,0,STON/O2. 3101282,7.9250,U,0,...,1,0,0,1,0,1,0,0,0,0
3,4,1.0,0,35.0,1,0,113803,53.1000,C123,0,...,1,1,0,0,0,0,0,1,0,0
4,5,0.0,1,35.0,0,0,373450,8.0500,U,0,...,1,0,0,1,0,0,1,0,0,0


In [122]:
cabinDf = pd.DataFrame()
full['Cabin'] = full['Cabin'].map(lambda c:c[0])#客舱首字母即为类别
full['Cabin'].head()

0    U
1    C
2    U
3    C
4    U
Name: Cabin, dtype: object

In [123]:
cabinDf = pd.get_dummies(full['Cabin'],prefix='Cabin')
full = pd.concat([full,cabinDf],axis=1)
full.drop('Cabin',axis=1,inplace=True)
full.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,...,Royalty,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,1,0.0,1,22.0,1,0,A/5 21171,7.2500,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,1.0,0,38.0,1,0,PC 17599,71.2833,1,0,...,0,0,0,1,0,0,0,0,0,0
2,3,1.0,0,26.0,0,0,STON/O2. 3101282,7.9250,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1.0,0,35.0,1,0,113803,53.1000,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,0.0,1,35.0,0,0,373450,8.0500,0,0,...,0,0,0,0,0,0,0,0,0,1


In [124]:
#存放家庭信息
familyDf = pd.DataFrame()
'''
家庭人数=同代直系亲属数（Parch）+不同代直系亲属数（SibSp）+乘客自己
（因为乘客自己也是家庭成员的一个，所以这里加1）
'''
familyDf[ 'FamilySize' ] = full[ 'Parch' ] + full[ 'SibSp' ] + 1

'''
家庭类别：
小家庭Family_Single：家庭人数=1
中等家庭Family_Small: 2<=家庭人数<=4
大家庭Family_Large: 家庭人数>=5
'''
#if 条件为真的时候返回if前面内容，否则返回0
familyDf[ 'Family_Single' ] = familyDf[ 'FamilySize' ].map( lambda s : 1 if s == 1 else 0 )
familyDf[ 'Family_Small' ]  = familyDf[ 'FamilySize' ].map( lambda s : 1 if 2 <= s <= 4 else 0 )
familyDf[ 'Family_Large' ]  = familyDf[ 'FamilySize' ].map( lambda s : 1 if 5 <= s else 0 )
familyDf.head()
full = pd.concat([full,familyDf],axis=1)
full.shape
full.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,FamilySize,Family_Single,Family_Small,Family_Large
0,1,0.0,1,22.0,1,0,A/5 21171,7.2500,0,0,...,0,0,0,0,0,1,2,0,1,0
1,2,1.0,0,38.0,1,0,PC 17599,71.2833,1,0,...,0,0,0,0,0,0,2,0,1,0
2,3,1.0,0,26.0,0,0,STON/O2. 3101282,7.9250,0,0,...,0,0,0,0,0,1,1,1,0,0
3,4,1.0,0,35.0,1,0,113803,53.1000,0,0,...,0,0,0,0,0,0,2,0,1,0
4,5,0.0,1,35.0,0,0,373450,8.0500,0,0,...,0,0,0,0,0,1,1,1,0,0


## 特征选择

In [125]:
corrDf = full.corr()
corrDf

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,FamilySize,Family_Single,Family_Small,Family_Large
PassengerId,1.000000,-0.005007,0.013406,0.025731,-0.055224,0.008942,0.031416,0.048101,0.011585,-0.049836,...,0.000549,-0.008136,0.000306,-0.045949,-0.023049,0.000208,-0.031437,0.028546,0.002975,-0.063415
Survived,-0.005007,1.000000,-0.543351,-0.070323,-0.035322,0.081629,0.257307,0.168240,0.003650,-0.149683,...,0.150716,0.145321,0.057935,0.016040,-0.026456,-0.316912,0.016639,-0.203367,0.279855,-0.125147
Sex,0.013406,-0.543351,1.000000,0.057397,-0.109609,-0.213125,-0.185484,-0.066564,-0.088651,0.115193,...,-0.057396,-0.040340,-0.006655,-0.083285,0.020558,0.137396,-0.188583,0.284537,-0.255196,-0.077748
Age,0.025731,-0.070323,0.057397,1.000000,-0.190747,-0.130872,0.171521,0.076179,-0.012718,-0.059153,...,0.132886,0.106600,-0.072644,-0.085977,0.032461,-0.271918,-0.196996,0.116675,-0.038189,-0.161210
SibSp,-0.055224,-0.035322,-0.109609,-0.190747,1.000000,0.373587,0.160224,-0.048396,-0.048678,0.073709,...,-0.015727,-0.027180,-0.008619,0.006015,-0.013247,0.009064,0.861952,-0.591077,0.253590,0.699681
Parch,0.008942,0.081629,-0.213125,-0.130872,0.373587,1.000000,0.221522,-0.008635,-0.100943,0.071881,...,-0.027385,0.001084,0.020481,0.058325,-0.012304,-0.036806,0.792296,-0.549022,0.248532,0.624627
Fare,0.031416,0.257307,-0.185484,0.171521,0.160224,0.221522,1.000000,0.286241,-0.130054,-0.169894,...,0.072737,0.073949,-0.037567,-0.022857,0.001179,-0.507197,0.226465,-0.274826,0.197281,0.170853
Embarked_C,0.048101,0.168240,-0.066564,0.076179,-0.048396,-0.008635,0.286241,1.000000,-0.164166,-0.778262,...,0.107782,0.027566,-0.020010,-0.031566,-0.014095,-0.258257,-0.036553,-0.107874,0.159594,-0.092825
Embarked_Q,0.011585,0.003650,-0.088651,-0.012718,-0.048678,-0.100943,-0.130054,-0.164166,1.000000,-0.491656,...,-0.061459,-0.042877,-0.020282,-0.019941,-0.008904,0.142369,-0.087190,0.127214,-0.122491,-0.018423
Embarked_S,-0.049836,-0.149683,0.115193,-0.059153,0.073709,0.071881,-0.169894,-0.778262,-0.491656,1.000000,...,-0.056023,0.002960,0.030575,0.040560,0.018111,0.137351,0.087771,0.014246,-0.062909,0.093671


In [126]:
"""
各特征对于Survived 的相关性排序
"""
corrDf['Survived'].sort_values(ascending = False)
#查看各个特征相对Survived的相关系数并排序

Survived         1.000000
Mrs              0.344935
Miss             0.332795
Pclass_1         0.285904
Family_Small     0.279855
Fare             0.257307
Cabin_B          0.175095
Embarked_C       0.168240
Cabin_D          0.150716
Cabin_E          0.145321
Cabin_C          0.114652
Pclass_2         0.093349
Master           0.085221
Parch            0.081629
Cabin_F          0.057935
Royalty          0.033391
Cabin_A          0.022287
FamilySize       0.016639
Cabin_G          0.016040
Embarked_Q       0.003650
PassengerId     -0.005007
Cabin_T         -0.026456
Officer         -0.031316
SibSp           -0.035322
Age             -0.070323
Family_Large    -0.125147
Embarked_S      -0.149683
Family_Single   -0.203367
Cabin_U         -0.316912
Pclass_3        -0.322308
Sex             -0.543351
Mr              -0.549199
Name: Survived, dtype: float64

In [127]:
"""
根据各个特征与生成情况（Survived）的相关系数大小，我们选择正负相关性较高的几个特征作为模型的输入：
头衔（前面所在的数据集titleDf）、客舱等级（pclassDf）、家庭大小（familyDf）、船票价格（Fare）、船舱号（cabinDf）、登船港口（embarkedDf）、性别（Sex）
"""

#构建特征子集
full_X = pd.concat( [titleDf,#头衔
                     pclassDf,#客舱等级
                     familyDf,#家庭大小
                     full['Fare'],#船票价格
                     cabinDf,#船舱号
                     embarkedDf,#登船港口
                     full['Sex']#性别
                    ] , axis=1 )
full_X.head()

,Master,Miss,Mr,Mrs,Officer,Royalty,Pclass_1,Pclass_2,Pclass_3,FamilySize,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S,Sex
0,0,0,1,0,0,0,0,0,1,2,...,0,0,0,0,0,1,0,0,1,1
1,0,0,0,1,0,0,1,0,0,2,...,0,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,1,0
3,0,0,0,1,0,0,1,0,0,2,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,1,1


## 建立训练数据集和测试数据集

In [130]:
'''
1）坦尼克号测试数据集因为是我们最后要提交给Kaggle的，里面没有生存情况的值，所以不能用于评估模型。
我们将Kaggle泰坦尼克号项目给我们的测试数据，叫做预测数据集（记为pred,也就是预测英文单词predict的缩写）。
也就是我们使用机器学习模型来对其生存情况就那些预测。
2）我们使用Kaggle泰坦尼克号项目给的训练数据集，做为我们的原始数据集（记为source），
从这个原始数据集中拆分出训练数据集（记为train：用于模型训练）和测试数据集（记为test：用于模型评估）。

'''
#原始数据集有891行
sourceRow=891
'''
sourceRow是我们在最开始合并数据前知道的，原始数据集有总共有891条数据
从特征集合full_X中提取原始数据集提取前891行数据时，我们要减去1，因为行号是从0开始的。
'''
#原始数据集：特征
source_X = full_X.loc[0:sourceRow-1,:]
#原始数据集：标签
source_y = full.loc[0:sourceRow-1,'Survived']   

#预测数据集：特征
pred_X = full_X.loc[sourceRow:,:]
'''
从原始数据集（source）中拆分出训练数据集（用于模型训练train），测试数据集（用于模型评估test）
train_test_split是交叉验证中常用的函数，功能是从样本中随机的按比例选取train data和test data
train_data：所要划分的样本特征集
train_target：所要划分的样本结果
test_size：样本占比，如果是整数的话就是样本的数量
'''
from sklearn.model_selection import train_test_split

#建立模型用的训练数据集和测试数据集
train_X, test_X, train_y, test_y = train_test_split(source_X ,
                                                    source_y,
                                                    train_size=.8)

#输出数据集大小
print ('原始数据集特征：',source_X.shape, 
       '训练数据集特征：',train_X.shape ,
      '测试数据集特征：',test_X.shape)

print ('原始数据集标签：',source_y.shape, 
       '训练数据集标签：',train_y.shape ,
      '测试数据集标签：',test_y.shape)

原始数据集特征： (891, 27) 训练数据集特征： (712, 27) 测试数据集特征： (179, 27)
原始数据集标签： (891,) 训练数据集标签： (712,) 测试数据集标签： (179,)


## 实现预测

In [131]:
from sklearn.linear_model import LogisticRegression#导入算法
model =LogisticRegression()#创建模型



In [132]:
model.fit(train_X,train_y)#传入训练数据

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

model.score(test_X,test_y)#模型准确度得分


0.8603351955307262

In [134]:
pred_Y = model.predict(pred_X)#使用我们创建的模型，对预测数据集中的生存情况进行预测
pred_Y = pred_Y.astype(int)#因为kaggle要求结果数据类型为int，我们进行数据类型转换
passengerId = full.loc[sourceRow:,'PassengerId']
#创建结果集
predDf = pd.DataFrame(
    {
        'PassengerId':passengerId,
        'Survived':pred_Y
    }
)
#导出结果
predDf.to_csv('titanic_pred.csv',index = False)
predDf.shape


(418, 2)